##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Context Caching Quickstart

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Caching.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook introduces context caching with the Gemini API and provides examples of interacting with the Apollo 11 transcript using the Python SDK. For a more comprehensive look, check out [the caching guide](https://ai.google.dev/gemini-api/docs/caching?lang=python).

### Install dependencies

In [1]:
%pip install -U -q "google-genai>=1.0.0"

In [2]:
from google import genai
from google.genai import types

### Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [3]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

## Upload a file

A common pattern with the Gemini API is to ask a number of questions of the same document. Context caching is designed to assist with this case, and can be more efficient by avoiding the need to pass the same tokens through the model for each new request.

This example will be based on the transcript from the Apollo 11 mission.

Start by downloading that transcript.

In [4]:
!wget -q https://storage.googleapis.com/generativeai-downloads/data/a11.txt
!head a11.txt

INTRODUCTION

This is the transcription of the Technical Air-to-Ground Voice Transmission (GOSS NET 1) from the Apollo 11 mission.

Communicators in the text may be identified according to the following list.

Spacecraft:
CDR	Commander	Neil A. Armstrong
CMP	Command module pilot   	Michael Collins
LMP	Lunar module pilot	Edwin E. ALdrin, Jr.


Now upload the transcript using the [File API](../quickstarts/File_API.ipynb).

In [5]:
document = client.files.upload(file="a11.txt")

## Cache the prompt

Next create a [`CachedContent`](https://ai.google.dev/api/python/google/generativeai/protos/CachedContent) object specifying the prompt you want to use, including the file and other fields you wish to cache. In this example the [`system_instruction`](../quickstarts/System_instructions.ipynb) has been set, and the document was provided in the prompt.

In [6]:
# Note that caching requires a frozen model, e.g. one with a `-001` suffix.
MODEL_ID="gemini-2.0-flash-001"  # @param ["gemini-2.0-flash-001", "gemini-1.5-flash-002","gemini-1.5-pro-002","gemini-1.5-flash-8b-latest"] {"allow-input":true, isTemplate: true}

apollo_cache = client.caches.create(
    model=MODEL_ID,
    config={
        'contents': [document],
        'system_instruction': 'You are an expert at analyzing transcripts.',
    },
)

apollo_cache

CachedContent(name='cachedContents/a8uq7cwrbfgx', display_name='', model='models/gemini-2.0-flash-001', create_time=datetime.datetime(2025, 4, 15, 18, 0, 21, 831586, tzinfo=TzInfo(UTC)), update_time=datetime.datetime(2025, 4, 15, 18, 0, 21, 831586, tzinfo=TzInfo(UTC)), expire_time=datetime.datetime(2025, 4, 15, 19, 0, 19, 197015, tzinfo=TzInfo(UTC)), usage_metadata=CachedContentUsageMetadata(audio_duration_seconds=None, image_count=None, text_count=None, total_token_count=322698, video_duration_seconds=None))

## Manage the cache expiry

Once you have a `CachedContent` object, you can update the expiry time to keep it alive while you need it.

In [7]:
client.caches.update(
    name=apollo_cache.name,
    config=types.UpdateCachedContentConfig(ttl="7200s")  # 2 hours in seconds
)
apollo_cache = client.caches.get(name=apollo_cache.name) # Get the updated cache
apollo_cache

CachedContent(name='cachedContents/a8uq7cwrbfgx', display_name='', model='models/gemini-2.0-flash-001', create_time=datetime.datetime(2025, 4, 15, 18, 0, 21, 831586, tzinfo=TzInfo(UTC)), update_time=datetime.datetime(2025, 4, 15, 18, 0, 31, 473887, tzinfo=TzInfo(UTC)), expire_time=datetime.datetime(2025, 4, 15, 20, 0, 31, 462272, tzinfo=TzInfo(UTC)), usage_metadata=CachedContentUsageMetadata(audio_duration_seconds=None, image_count=None, text_count=None, total_token_count=322698, video_duration_seconds=None))

## Use the cache for generation

As the `CachedContent` object refers to a specific model and parameters, you must create a [`GenerativeModel`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) using [`from_cached_content`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#from_cached_content). Then, generate content as you would with a directly instantiated model object.

In [14]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents='Find a lighthearted moment from this transcript',
    config=types.GenerateContentConfig(
        cached_content=apollo_cache.name,
    )
)

print(response.text)

Here's a lighthearted exchange between the crew and Houston:

00 05 20 31 CMP
If we're late in answering you, it's because we're munching sandwiches.

00 05 20 36 CC
Roger. I wish I could do the same here.

00 05 20 40 CMP
No. Don't leave the console!

00 05 20 42 CC
Don't worry. I won't.

00 05 20 47 CMP
FLIGHT doesn't like it.

00 05 20 54 CMP
How is FLIGHT today?

00 05 20 58 CC
Oh, he's doing quite well.

This exchange is humorous because it's a relatable moment of snacking while working, with the crew teasing Houston about taking breaks, and ending with an inquiry about Flight Director's well-being.



You can inspect token usage through `usage_metadata`. Note that the cached prompt tokens are included in `prompt_token_count`, but excluded from the `total_token_count`.

In [15]:
response.usage_metadata

GenerateContentResponseUsageMetadata(cache_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=322698)], cached_content_token_count=322698, candidates_token_count=225, candidates_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=225)], prompt_token_count=322706, prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=322706)], thoughts_token_count=None, tool_use_prompt_token_count=None, tool_use_prompt_tokens_details=None, total_token_count=322931, traffic_type=None)

You can ask new questions of the model, and the cache is reused.

In [16]:
chat = client.chats.create(
  model = MODEL_ID,
  config = {"cached_content": apollo_cache.name}
)

response = chat.send_message(message="Give me a quote from the most important part of the transcript.")
print(response.text)

Based on the provided transcript, the most important part is when Neil Armstrong first sets foot on the moon. His quote is:

"THAT'S ONE SMALL STEP FOR (A) MAN, ONE GIANT LEAP FOR MANKIND."



In [17]:
response = chat.send_message(
    message="What was recounted after that?",
    config = {"cached_content": apollo_cache.name}
)
print(response.text)

Immediately after Neil Armstrong's famous quote, the following details were recounted:

*   He describes the surface as "fine and powdery" that adheres to his boots "like powdered charcoal."
*   He sinks into the surface "a small fraction of an inch," seeing the footprints of his boots in the fine, sandy particles.
*   He finds "no difficulty in moving around," and finds it even easier than in simulations.
*   He states that the descent engine did not leave a crater.
*   He notes that the landing area is level.
*   He sees "evidence of rays emanating from the descent engine, but a very insignificant amount."


In [18]:
response.usage_metadata

GenerateContentResponseUsageMetadata(cache_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=322698)], cached_content_token_count=322698, candidates_token_count=136, candidates_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=136)], prompt_token_count=322766, prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=322766)], thoughts_token_count=None, tool_use_prompt_token_count=None, tool_use_prompt_tokens_details=None, total_token_count=322902, traffic_type=None)

From the metadata, you can see that the `total_token_count=322902` and `cached_content_token_count=322698`, and `candidates_token_count=136`.

This means that from the 322902 tokens, 322698 were cached (and thus less expensive) and only 136 were from the prompt.

Since the cached tokens are cheaper than the normal ones, it means this prompt was much cheaper that if you had not used caching. Check the [pricing here](https://ai.google.dev/pricing) for the up-to-date discount on cached tokens.

## Delete the cache

The cache has a small recurring storage cost (cf. [pricing](https://ai.google.dev/pricing)) so by default it is only saved for an hour. In this case you even set it up for a shorter amont of time (using `"ttl"`) of 2h.

Still, if you don't need you cache anymore, it is good practice to delete it proactively.

In [19]:
print(apollo_cache.name)
client.caches.delete(name=apollo_cache.name)

cachedContents/a8uq7cwrbfgx


DeleteCachedContentResponse()

## Next Steps
### Useful API references:

If you want to know more about the caching API, you can check the full [API specifications](https://ai.google.dev/api/rest/v1beta/cachedContents) and the [caching documentation](https://ai.google.dev/gemini-api/docs/caching).

### Continue your discovery of the Gemini API

Check the File API notebook to know more about that API. The [vision capabilities](../quickstarts/Video.ipynb) of the Gemini API are a good reason to use the File API and the caching.
The Gemini API also has configurable [safety settings](../quickstarts/Safety.ipynb) that you might have to customize when dealing with big files.
